In [33]:
import pandas as pd
import re
import os
import json
import time
import requests
from datetime import datetime
from bs4 import BeautifulSoup
import http.client, urllib.parse

In [34]:
from requests import get
url = "https://www.espn.com/mlb/history/leaders/_/breakdown/season/year/2022"
page = requests.get(url)

In [35]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(response.content, "html.parser")

In [36]:
print(soup.prettify())


<!DOCTYPE html>
<html xmlns:fb="https://www.facebook.com/2008/fbml">
 <head>
  <script>
   (function redirectToHttpIfHttps() {
   var win      = typeof window !== 'undefined' && window,
       location = win && win.location,
       protocol = location && location.protocol;

   if (protocol === 'https:' && !true) {
        location.href = location.href.replace('https://', 'http://');
   }
})();
  </script>
  <meta charset="utf-8"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <link href="https://a.espncdn.com/favicon.ico" mask="" rel="icon" sizes="any"/>
  <meta content="#CC0000" name="theme-color"/>
  <script type="text/javascript">
   if(true && navigator && navigator.userAgent.toLowerCase().indexOf("teamstream") >= 0) {
        window.location = 'http://m.espn.com/mobilecache/general/apps/sc';
    }
  </script>
  <title>
   MLB Baseball Career Batting Leaders - Major League Baseball - ESPN
  </title>
  <meta content="xuj1ODRluWa0frM-BjIr_aSHoUC7HB5C1MgmYAM_GkA"

In [37]:
soup.find_all('tr', attrs={'class':'colhead'})

[<tr align="right" class="colhead"><td></td><td align="left" width="30%">PLAYER</td><td align="left">YRS</td><td><a href="//www.espn.com/mlb/history/leaders/_/breakdown/season/year/2022/sort/gamesPlayed" title="Games">G</a></td><td><a href="//www.espn.com/mlb/history/leaders/_/breakdown/season/year/2022/sort/atBats" title="At Bats">AB</a></td><td><a href="//www.espn.com/mlb/history/leaders/_/breakdown/season/year/2022/sort/runs" title="Runs Scored">R</a></td><td><a href="//www.espn.com/mlb/history/leaders/_/breakdown/season/year/2022/sort/hits" title="Hits">H</a></td><td><a href="//www.espn.com/mlb/history/leaders/_/breakdown/season/year/2022/sort/doubles" title="Doubles">2B</a></td><td><a href="//www.espn.com/mlb/history/leaders/_/breakdown/season/year/2022/sort/triples" title="Triples">3B</a></td><td><a href="//www.espn.com/mlb/history/leaders/_/breakdown/season/year/2022/sort/homeRuns" title="Home Runs">HR</a></td><td><a href="//www.espn.com/mlb/history/leaders/_/breakdown/season/ye

In [40]:
header = soup.find('tr', attrs={'class': 'colhead'})
columns = [col.get_text() for col in header.find_all('td')]
final_df = pd.DataFrame(columns=columns)
final_df

,,PLAYER,YRS,G,AB,R,H,2B,3B,HR,RBI,BB,SO,SB,CS,BA


In [45]:
players = soup.find_all('tr', attrs={'class':re.compile('row player-10-')})
for player in players:
    stats = [stat.get_text() for stat in player.find_all('td')]
    temp_df = pd.DataFrame(stats).transpose()
    temp_df.columns = columns
    final_df  = pd.concat([final_df,temp_df], ignore_index=True)

In [46]:
for i in range (1,300,50):
  url = 'https://www.espn.com/mlb/history/leaders/_/breakdown/season/year/2022'.format(i)
  page = requests.get(url)
  soup = BeautifulSoup(page.text,'html.parser')
  players = soup.find_all('tr', attrs={'class':re.compile('row player-10-')})

  for player in players:
    stats = [stat.get_text() for stat in player.find_all('td')]
    temp_df = pd.DataFrame(stats).transpose()
    temp_df.columns = columns
    final_df  = pd.concat([final_df,temp_df], ignore_index=True)
final_df

,,PLAYER,YRS,G,AB,R,H,2B,3B,HR,RBI,BB,SO,SB,CS,BA
0,1,Jeff McNeil,2,148,533,73,174,39,1,9,62,40,61,4,0,.326
1,2,Freddie Freeman,10,159,612,117,199,47,2,21,100,84,102,13,3,.325
2,3,Joey Meneses,0,56,222,33,72,14,0,13,34,15,52,1,0,.324
3,4,Paul Goldschmidt,9,151,561,106,178,41,0,35,115,79,141,7,0,.317
4,5,Luis Arraez,1,144,547,88,173,31,1,8,49,50,43,4,4,.316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,46,Jose Ramirez,7,157,601,90,168,44,5,29,126,69,82,20,7,.280
346,,Alex Verdugo,3,152,593,75,166,39,1,11,74,42,86,1,3,.280
347,,Alec Bohm,0,152,586,79,164,24,3,13,72,31,110,2,3,.280
348,49,Justin Turner,0,128,468,61,130,36,0,13,81,50,89,3,0,.278


In [48]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/mlb2022stats.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
final_df.to_csv(f)

Mounted at /content/drive
